[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/predict.ipynb)

In [1]:
!pip install lightgbm

In [2]:
!pip install --upgrade xgboost

Requirement already up-to-date: xgboost in /usr/local/lib/python3.6/dist-packages (0.80)


In [3]:
from google.colab import drive
from google.colab import files

import pickle

import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb
from keras import models
from keras import layers
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import mean_squared_error


Using TensorFlow backend.


In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
%cd ../content/gdrive/My Drive/projects/ggl_an/

/content/gdrive/My Drive/projects/ggl_an


In [6]:
%ls

model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [7]:
%%time
df_user_train = pd.read_csv('user_train_n.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_user_test = pd.read_csv('user_test_n.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

df_y = df_user_train['transactionRevenue_sum']
df_x = df_user_train.drop(['transactionRevenue_sum'], axis=1)
df_tgt = df_user_test.drop(['transactionRevenue_sum'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state=1)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test, random_state=1)

CPU times: user 15.4 s, sys: 1.69 s, total: 17.1 s
Wall time: 17.7 s


In [8]:
%%time
models = []

filename = 'model_LGB.sav'
model_LGB = pickle.load(open(filename, 'rb'))
models.append(model_LGB)

filename = 'model_XGB.sav'
model_XGB = pickle.load(open(filename, 'rb'))
models.append(model_XGB)

filename = 'model_NN.h5'
model_NN = load_model(filename)
models.append(model_NN)



CPU times: user 883 ms, sys: 140 ms, total: 1.02 s
Wall time: 1.02 s


In [9]:
for model in models:
    y_pred = model.predict(x_valid.fillna(0))
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)
    print(rmse)

1.9611630374254396
2.0692074389461808
2.400328732028105


In [10]:
%%time
preds = []
for model in models:
    pred_tmp = model.predict(df_tgt)
    preds.append(pred_tmp)

CPU times: user 41.5 s, sys: 3.28 s, total: 44.8 s
Wall time: 29.8 s


In [11]:
%%time
df_preds = pd.DataFrame(preds).T

CPU times: user 32.3 s, sys: 355 ms, total: 32.7 s
Wall time: 32.7 s


In [17]:
%%time
pred_tgt = df_preds.sum(axis=1)/len(preds)
pred_tgt[pred_tgt<0] = 0
pred_tgt = pred_tgt.values.tolist()

CPU times: user 1.7 s, sys: 69.6 ms, total: 1.77 s
Wall time: 1.77 s


In [0]:
df_tgt['PredictedLogRevenue'] = pred_tgt

In [0]:
df_tgt[['PredictedLogRevenue']].to_csv("output.csv", index=True)

In [22]:
df_output = pd.read_csv('output.csv')
df_output.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.130107
1,0000049363351866189,0.002159
2,0000053049821714864,0.002159
3,0000059488412965267,0.415486
4,0000085840370633780,0.002846


In [23]:
type(df_output['fullVisitorId'][0])

str

In [24]:
df_output.shape

(617242, 2)

In [25]:
df_output.describe()

,PredictedLogRevenue
count,617242.000000
mean,0.244373
std,1.462071
min,0.000000
25%,0.002159
50%,0.002159
75%,0.002846
max,148.785380
